## Tokenizer

In [7]:
from nltk.tokenize import TweetTokenizer
tokenizer = TweetTokenizer(preserve_case=False)

In [86]:
import torch, torchtext
import torch.utils.data as tdata

from torchtext.datasets import text_classification

In [9]:
device = torch.device("cuda:1")

## Data load

In [13]:
import pandas as pd
from pandas import DataFrame, Series
import numpy as np


f_s = "/home/bwlee/data/yelp_review_polarity_csv/reviews_with_splits_full.csv"
df = pd.read_csv(f_s, header=0)
label0 = []
for i in df.index: # change this for getting info based on previous day market move
    if df['rating'].iloc[i] == 'positive':
        label0.append(1)
    else:
        label0.append(0)
df['rating'] = label0

In [24]:
train_df = df[df['split']=='train'][['rating', 'review']]
val_df = df[df['split']=='val'][['rating', 'review']]
test_df = df[df['split']=='test'][['rating', 'review']]
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

In [29]:
train_df.head()

,rating,review
0,0,the entrance was the impressive thing about th...
1,0,"i m a mclover , and i had no problem nwith the..."
2,0,"less than good here , not terrible , but i see..."
3,0,i don t know if i can ever bring myself to go ...
4,0,food was ok good but the service was terrible ...


## Set data field

In [35]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [38]:
from torchtext import data
# If use_vocab is True, it should have vocabulary 
# when Dataloader is iterating
# It can be generated by calling build_vocab
TEXT = data.Field(sequential=True, use_vocab=True,
                 tokenize='spacy', lower=True,
                 batch_first=True, fix_length=100
                 )

LABEL = data.Field(sequential=False, use_vocab=False,
                  batch_first=False, is_target=True)

## Make dataset

In [145]:
from torchtext.data import Dataset, Batch, Example
from torchtext.data import Iterator, BucketIterator

In [119]:
class DFDataset(Dataset):
    """
    Modified TabularDataset for load from dataframe
    """
    def __init__(self, train_df, fields):
        super().__init__(train_df, fields)
        self.rating = train_df['rating']
        self.review = train_df['review']
        self.rating = self.rating.reset_index(drop=True)
        self.review = self.review.reset_index(drop=True)

    def __len__(self):
        return len(self.rating)
    
    def __getitem__(self, ii):
        return (self.rating[ii], self.review[ii])

In [129]:
train_df.iloc[:1000].to_csv('temp.csv', index=False)

In [131]:
from torchtext.data import TabularDataset

In [138]:
train_data = TabularDataset(path='temp.csv', format='csv', fields=(
        ('rating', LABEL), ('review', TEXT)) )
TEXT.build_vocab(train_data, min_freq=10, max_size=10000)

In [151]:
len(train_df)

392000

In [153]:
train_df0 = train_df[:1000]
fields0 = (('rating', LABEL), ('review', TEXT))
train_data = [ Example.fromlist(
    train_df0.iloc[ii].values.tolist(),
                              fields0)
              for ii in range(len(train_df0)) ]
#train_data = Dataset(train_df.values.tolist(), fields=(

#train_data = TextDataset(train_df, fields=(
train_data = Dataset(train_data, fields=fields0)

In [76]:
#train_iter = BucketIterator.splits((train_df), batch_size=3)

In [154]:
iter1 = Iterator(train_data, batch_size=10)

In [155]:
batch = next(iter(iter1))

In [156]:
batch.review

tensor([[ 306,   16,  320,    6,    5,   82,  929,    2,    0,    5,   40,   72,
           20,   79,  134,   55,    2,    2,    2,   41,  137,    2,    5,   96,
           21,  174,   84,  346,   11,    3,  218,   82,   68,   41,   19,   23,
           17,   28, 1204,   54,    2,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1],
        [  19,   39,  295,    7,   36,   80,   56,    2,   22,  155,    3,  182,
         1091,   84,   22,   88,   20,  135,  691,    6,  615, 1176,    2,    3,
          615, 1176,  127,  574,  142,    0,   25,   72,  268,   22,   47,    0,
            0,    2,   22,   31,    0,   66,  331,   84,   18,  418,    0, 

In [87]:
dataloader = tdata.DataLoader(train_data, batch_size=3)

In [ ]:
for x, y in dataloader:
    print(x, y)

In [85]:
for i, x in enumerate(iter1):
    print(x)
    if i>0: break

AttributeError: 'list' object has no attribute 'rating'

In [73]:
train_loader = Iterator(dataset=train_data, batch_size=5)
batch = next(iter(train_loader))

AttributeError: 'list' object has no attribute 'rating'

In [52]:
train_data

In [77]:
train_txt, val_txt, test_txt = torchtext.datasets.WikiText2.splits(TEXT)

downloading wikitext-2-v1.zip


wikitext-2-v1.zip: 100%|██████████| 4.48M/4.48M [00:02<00:00, 2.01MB/s]


extracting


In [78]:
train_txt